## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hot Springs,US,34.5037,-93.0552,72.00,89,90,4.00,overcast clouds
1,1,Lashio,MM,22.9333,97.7500,66.56,97,73,2.21,broken clouds
2,2,Mossendjo,CG,-2.9506,12.7261,83.46,60,99,3.60,overcast clouds
3,3,Avera,US,33.1940,-82.5271,52.72,75,69,5.88,broken clouds
4,4,Te Anau,NZ,-45.4167,167.7167,50.95,98,93,1.01,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input('What is the minimum temperature you would like for your trip? '))
max_temp=float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df['Max Temp']>=min_temp)&(city_data_df['Max Temp']<=max_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hot Springs,US,34.5037,-93.0552,72.00,89,90,4.00,overcast clouds
2,2,Mossendjo,CG,-2.9506,12.7261,83.46,60,99,3.60,overcast clouds
7,7,Kirando,TZ,-7.4167,30.6000,86.77,52,61,9.73,broken clouds
8,8,Kiunga,PG,-6.1219,141.2906,75.16,99,100,5.08,light rain
9,9,Eyl,SO,7.9803,49.8164,83.66,62,5,16.28,clear sky
...,...,...,...,...,...,...,...,...,...,...
723,723,Tabou,CI,4.4230,-7.3528,79.03,86,98,6.11,moderate rain
725,725,Thinadhoo,MV,0.5333,72.9333,83.55,69,28,14.61,scattered clouds
727,727,Oussouye,SN,12.4850,-16.5469,87.98,66,97,5.26,overcast clouds
728,728,Padang,ID,-0.9492,100.3543,77.05,90,95,2.93,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df[preferred_cities_df.isna().any(axis=1)]

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
219,219,Luderitz,NaN,-26.6481,15.1594,74.88,44,0,24.83,clear sky


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df=preferred_cities_df.dropna()

In [7]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hot Springs,US,72.00,overcast clouds,34.5037,-93.0552,
2,Mossendjo,CG,83.46,overcast clouds,-2.9506,12.7261,
7,Kirando,TZ,86.77,broken clouds,-7.4167,30.6000,
8,Kiunga,PG,75.16,light rain,-6.1219,141.2906,
9,Eyl,SO,83.66,clear sky,7.9803,49.8164,
12,Gamba,GA,78.80,overcast clouds,-2.6500,10.0000,
13,Hermanus,ZA,71.91,scattered clouds,-34.4187,19.2345,
18,Morant Bay,JM,80.22,scattered clouds,17.8815,-76.4093,
20,Cape Town,ZA,72.12,clear sky,-33.9258,18.4232,
24,Kapaa,US,75.52,broken clouds,22.0752,-159.3190,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.

#dictionary to drop the rows with no Hotel Name.
hotels_no_name=[]

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat= row['Lat']
    lng=row['Lng']
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6c. Get latitude and longitude from DataFrame
    hotel_df[["Lat", "Lng"]]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        pass
        print('IndexError',index)
        #indexes rows to be dropped with no Hotel Name
        if index not in hotels_no_name:
            hotels_no_name.append(index)

print(hotels_no_name)
hotel_df

IndexError 12
IndexError 108
IndexError 151
IndexError 153
IndexError 240
IndexError 305
IndexError 323
IndexError 354
IndexError 359
IndexError 362
IndexError 411
IndexError 430
IndexError 513
IndexError 533
IndexError 598
IndexError 613
IndexError 620
IndexError 640
IndexError 665
IndexError 717
IndexError 725
[12, 108, 151, 153, 240, 305, 323, 354, 359, 362, 411, 430, 513, 533, 598, 613, 620, 640, 665, 717, 725]


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hot Springs,US,72.00,overcast clouds,34.5037,-93.0552,Embassy Suites by Hilton Hot Springs Hotel & Spa
2,Mossendjo,CG,83.46,overcast clouds,-2.9506,12.7261,HOTEL D'ASSISE
7,Kirando,TZ,86.77,broken clouds,-7.4167,30.6000,Lakeshore Lodge
8,Kiunga,PG,75.16,light rain,-6.1219,141.2906,Airport Lodge Kiunga
9,Eyl,SO,83.66,clear sky,7.9803,49.8164,Amiin Hotel
...,...,...,...,...,...,...,...
723,Tabou,CI,79.03,moderate rain,4.4230,-7.3528,hotêl doufoulougou
725,Thinadhoo,MV,83.55,scattered clouds,0.5333,72.9333,
727,Oussouye,SN,87.98,overcast clouds,12.4850,-16.5469,Auberge du Routard
728,Padang,ID,77.05,light rain,-0.9492,100.3543,Mercure Padang


In [10]:
#Eventhough exists no Hotels Names, the DF shows no null rows.
hotel_df[hotel_df['Hotel Name'].isnull()]

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.drop(hotels_no_name, inplace=True)
clean_hotel_df=hotel_df

In [12]:
# 8a. Create the output File (CSV)
output_data_file=('WeatherPy_vacation.csv')

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and Max. Temp. of: {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map.

locations = clean_hotel_df["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(50.0, 31.0), zoom_level=1)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))